In [1]:
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

import os
import h5py
import librosa
import itertools
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import scipy

import keras
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
def split_song(X, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    shape = X.shape[0]
    chunk = int(shape*window)
    offset = int(chunk*(1.-overlap))
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, shape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)

    return np.array(temp_X)

In [3]:
def extract_melspectrogram(songs):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft = 1024, hop_length = 512)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    return np.array(list(tsongs))

In [13]:
def read_data(src_dir, song_samples):
    # Empty array of dicts with the processed features from all files
    arr_specs = []
    arr_names = []

    # Read files from the folders
    for root, subdirs, files in os.walk(src_dir):
        for file in files:
            # Read the audio file
            file_name = src_dir + "/" + file
            print("Read file: {}".format(file))

            signal, sr = librosa.load(file_name, offset=30.0, duration=30.0)
            signal = signal[:song_samples]

            # Convert to dataset of spectograms/melspectograms
            signals = split_song(signal)

            # Convert to "spec" representation
            specs = extract_melspectrogram(signals)
            # Save files
            arr_specs.append(specs)
            arr_names.append(file)
    return np.array(arr_specs), np.array(arr_names)

In [14]:
def predict_genres(songs, models_name):

    model = load_model(models_name)

    predicts = []
    for X in songs:
        predicts.append(model.predict(X))

    return np.array(predicts)

In [15]:
def normalize_data(predicts):

    songs = []
    
    #normalize the results
    for song_predict in predicts:
        i = 0
        temp_separate = np.zeros((10, 19))
        for sample in song_predict:
            j = 0
            for x in sample:
                temp_separate[j][i] = x
                j = j + 1
            i = i + 1
        
        genres_temp = []
        for x in temp_separate:
            genres_temp.append(scipy.mean(x))
            
        songs.append(genres_temp)

    return np.array(songs)

In [16]:
song_samples = 660000
test_folder = 'data/test'
songs_sample, file_names = read_data(test_folder, song_samples)
print(songs_sample.shape)


Read file: 001. Imagine Dragons - Natural.mp3
Read file: 002. Imagine Dragons - Believer.mp3
Read file: 003. Coldplay - Viva La Vida.mp3
Read file: 004. Imagine Dragons - Radioactive (из фильма «Гостья»).mp3
Read file: 005. Queen - The Show Must Go On (Remastered 2011).mp3
Read file: 006. Red Hot Chili Peppers - Californication.mp3
Read file: 007. Limp Bizkit - Behind Blue Eyes (The Who cover).mp3
Read file: 008. Queen - Bohemian Rhapsody (Remastered 2011).mp3
Read file: 009. Bring Me The Horizon, Dani Filth, Simon Dobson, Ch - wonderful life.mp3
Read file: 01. Монеточка - Каждый раз.mp3
Read file: 010. Evanescence - Bring Me To Life.mp3
Read file: 011. Red Hot Chili Peppers - Snow (Hey Oh).mp3
Read file: 012. Nirvana - Smells Like Teen Spirit.mp3
Read file: 013. OneRepublic - Counting Stars.mp3
Read file: 014. Saliva - They Don't Care About Us.mp3
Read file: 015. Queen - Another One Bites The Dust (Remastered 2011).mp3
Read file: 016. Amy Macdonald - This Is The Life.mp3
Read file: 01

Read file: 54. Егор Крид - Миллион алых роз.mp3
Read file: 55. Ed Sheeran - Shape of You.mp3
Read file: 56. Burak Yeter  Cecilia Krull - My Life Is Going On (Burak Yeter Remix).mp3
Read file: 57. MO  Diplo - Sun In Our Eyes.mp3
Read file: 58. Тимати feat. Мот  Егор Крид  Скруджи  НАZИМА  TERRY - Ракета.mp3
Read file: 59. Тимати feat. Егор Крид - Гучи.mp3
Read file: 60. ZAYN - Sour Diesel.mp3
Read file: 61. Зомб - Она горит на танцполе.mp3
Read file: 62. Артур Пирожков - Чика.mp3
Read file: 63. Loboda - случайная.mp3
Read file: 64. Макс Барских - Туманы.mp3
Read file: 65. Sergey Lazarev - Sdavaysya.mp3
Read file: 66. БИ-2 feat. Oxxxymiron - Пора возвращаться домой.mp3
Read file: 67. Loboda - Парень.mp3
Read file: 68. Баста - Сансара (feat. Диана Арбенина  Александр Ф. Скляр  Сергей Бобунец  Sunsa.mp3
Read file: 69. The White Stripes - Seven Nation Army.mp3
Read file: 70. Loboda - Лети.mp3
Read file: 71. Addal vs Mida ft. KiFi - High (Radio Edit).mp3
Read file: 72. CYGO - Panda E.mp3
Rea

In [17]:
models_name = 'models/model_snn.h5'
predicts = predict_genres(songs_sample, models_name)
print(predicts.shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
(210, 19, 10)


In [18]:
import pandas as pd 


genres = ['metal', 'disco', 'classical', 'hiphop', 'jazz', 
           'country', 'pop', 'blues', 'reggae', 'rock']

songs = normalize_data(predicts)
song_matrix = pd.DataFrame(data=songs,
                  index=file_names,
                  columns=genres
                 )
song_matrix

,metal,disco,classical,hiphop,jazz,country,pop,blues,reggae,rock
001. Imagine Dragons - Natural.mp3,2.103849e-02,2.628970e-02,5.466077e-02,0.129646,2.477771e-01,1.299704e-01,0.200526,7.847802e-02,0.017095,9.451817e-02
002. Imagine Dragons - Believer.mp3,2.720439e-02,2.065134e-02,1.746000e-08,0.561530,4.203129e-03,6.177597e-03,0.158792,3.427952e-02,0.071800,1.153611e-01
003. Coldplay - Viva La Vida.mp3,2.611399e-03,3.864851e-01,2.682056e-03,0.000255,2.740217e-03,1.984843e-01,0.052827,5.502416e-03,0.005097,3.433154e-01
004. Imagine Dragons - Radioactive (из фильма «Гостья»).mp3,5.741810e-02,1.086855e-02,2.580333e-06,0.184211,1.130291e-02,7.611868e-03,0.381051,1.327891e-03,0.182015,1.641911e-01
005. Queen - The Show Must Go On (Remastered 2011).mp3,2.139438e-02,5.378156e-02,1.087782e-02,0.005988,4.385200e-02,2.307867e-01,0.199754,2.361982e-02,0.022780,3.871657e-01
006. Red Hot Chili Peppers - Californication.mp3,2.659467e-03,9.399283e-03,4.653287e-10,0.007707,2.230345e-02,2.379127e-01,0.059743,3.795385e-01,0.046429,2.343070e-01
007. Limp Bizkit - Behind Blue Eyes (The Who cover).mp3,1.079112e-03,7.372202e-03,4.962957e-02,0.000529,9.346462e-02,6.002834e-01,0.073964,4.793140e-02,0.005769,1.199771e-01
008. Queen - Bohemian Rhapsody (Remastered 2011).mp3,5.089555e-02,4.364955e-02,5.970999e-02,0.020993,2.527389e-01,1.579276e-01,0.031407,7.791839e-02,0.041953,2.628068e-01
"009. Bring Me The Horizon, Dani Filth, Simon Dobson, Ch - wonderful life.mp3",9.380557e-01,7.533723e-05,6.348831e-09,0.004056,7.972132e-06,1.453697e-06,0.036452,2.048432e-06,0.000100,2.125035e-02
01. Монеточка - Каждый раз.mp3,4.582061e-06,9.838353e-02,8.262352e-24,0.086920,5.151569e-13,1.615521e-06,0.737296,8.346639e-05,0.075508,1.802446e-03


In [19]:
song_matrix.to_csv('songs.csv')